# Visual odometry pipeline

### Import requirements

In [1]:
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np

%matplotlib ipympl

### Load images and depth map

In [2]:
folderChoice = 0
path = "".join(["testImages/visualOdometryTestImages/", str(folderChoice)])

# Load set of top images
imageGlobT = sorted(glob.glob("".join([path, "/top_*", ".png"])))

# Load set of bottom images
imageGlobB = sorted(glob.glob("".join([path, "/bottom_*", ".png"])))

# Load depth map
imageGlobD = sorted(glob.glob("".join([path, "/topDepth_*", ".png"])))

if not (len(imageGlobT)==len(imageGlobB) and \
                len(imageGlobB)==len(imageGlobD)):
    print("Images could not be matched")

print ("Selections: 0-{}".format(len(imageGlobT)-1))

Selections: 0-1


In [3]:
viewSelection = 0

# Top images
imageT0 = cv2.imread(imageGlobT[viewSelection])
imageT1 = cv2.imread(imageGlobT[viewSelection+1])

# Bottom images
imageB0 = cv2.imread(imageGlobB[viewSelection])
imageB1 = cv2.imread(imageGlobB[viewSelection+1])

# Depth maps; -1 flag to load them as is
depth0 = cv2.imread(imageGlobD[viewSelection], -1)
depth1 = cv2.imread(imageGlobD[viewSelection+1], -1)

### View top camera images

In [4]:
plt.figure()
plt.suptitle("Top images")
plt.imshow(cv2.cvtColor(np.hstack([imageT0, imageT1]), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### View bottom camera images

In [5]:
plt.figure()
plt.suptitle("Bottom images")
plt.imshow(cv2.cvtColor(np.hstack([imageB0, imageB1]), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### View depth images
Depth images correspond to top camera

In [6]:
plt.figure()
plt.suptitle("Depth maps")
plt.imshow(np.hstack([depth0, depth1]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …